In [11]:
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import geopandas as gpd
from shapely.geometry import Point

In [66]:
arrays = dict(np.load("Data//transactions.npz"))

data = {k: [s.decode("utf-8") for s in v.tobytes().split(b"\x00")] if v.dtype == np.uint8 else v for k, v in arrays.items()}
transactions = pd.DataFrame.from_dict(data)

KeyboardInterrupt: 

In [3]:
#On retient que les transaction depuis le 1er janvier 2019
transactions_2019=transactions[(transactions['type_batiment']=='Appartement')&(transactions['date_transaction']>dt.datetime(2018, 12, 31))][['id_transaction','date_transaction','prix','departement','id_ville','adresse','ville','code_postal','latitude', 'longitude','type_batiment','vefa','n_pieces','surface_habitable']]
#transactions_2019['Quarter']=transactions_2019.date_transaction.apply(lambda x: x.to_period('Q'))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\VLAGO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-6a1d032a3ca1>", line 2, in <module>
    transactions_2019=transactions[(transactions['type_batiment']=='Appartement')&(transactions['date_transaction']>dt.datetime(2018, 12, 31))][['id_transaction','date_transaction','prix','departement','id_ville','adresse','ville','code_postal','latitude', 'longitude','type_batiment','vefa','n_pieces','surface_habitable']]
  File "C:\Users\VLAGO\anaconda3\lib\site-packages\pandas\core\frame.py", line 2791, in __getitem__
    return self._getitem_bool_array(key)
  File "C:\Users\VLAGO\anaconda3\lib\site-packages\pandas\core\frame.py", line 2845, in _getitem_bool_array
    return self._take_with_is_copy(indexer, axis=0)
  File "C:\Users\VLAGO\anaconda3\lib\site-packages\pandas\core\generic.py", line 3409, in _take_with_is_copy
    re

KeyboardInterrupt: 

In [12]:
transactions_2019=transactions_2019[['id_transaction','latitude','longitude']]
#transactions_2019.to_csv('transactions_2019_prepro.csv',sep=';',index=False)

In [15]:
#transactions_2019=pd.read_csv('transactions_2019_prepro.csv',sep=';',index_col=0)
transactions_2019.head()

C:\Users\VLAGO\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id_transaction,latitude,longitude
35884,107332,45.903703,4.814195
35889,115805,46.163569,5.566400
35893,113527,46.308764,5.945066
35895,104202,45.849502,5.434411
35897,106853,45.826389,4.981686


In [2]:
geoiris = gpd.read_file("Data\\iris-2013-01-01\\iris-2013-01-01.shp")
geoiris.head()

,DEPCOM,NOM_COM,IRIS,DCOMIRIS,NOM_IRIS,TYP_IRIS,ORIGINE,geometry
0,97201,L' AJOUPA-BOUILLON,0000,972010000,L'AJOUPA-BOUILLON,Z,None,"POLYGON ((-61.09797 14.83121, -61.09867 14.830..."
1,97202,LES ANSES-D'ARLET,0000,972020000,LES ANSES-D'ARLET,Z,None,"POLYGON ((-61.07379 14.54041, -61.07337 14.538..."
2,97203,BASSE-POINTE,0000,972030000,BASSE-POINTE,Z,None,"POLYGON ((-61.08307 14.85248, -61.08441 14.850..."
3,97204,LE CARBET,0000,972040000,LE CARBET,Z,None,"POLYGON ((-61.15649 14.73308, -61.15488 14.733..."
4,97205,CASE-PILOTE,0000,972050000,CASE-PILOTE,Z,None,"POLYGON ((-61.12423 14.69021, -61.12086 14.689..."


In [28]:
#jointure spatial entre les coordonnées et les contours IRIS

def batch_iris(df,geo):
    
    #geoiris = gpd.read_file("Data\\iris-2013-01-01\\iris-2013-01-01.shp")

    crs = {'init': 'epsg:4326'}
    geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
    geo_transaction = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

    return gpd.sjoin(geo_transaction, geo)
    

In [29]:
#on fait des batch de 20 000 pour éviter de saturer la mémoire

tt=np.shape(transactions_2019)[0]
size_batch=20000
batch=tt//size_batch

for i in range(batch):    
    df=transactions_2019.iloc[i*size_batch:size_batch*(i+1)]
    res=batch_iris(df,geoiris)
    res=res[['id_transaction','latitude','longitude','geometry','DEPCOM','NOM_COM','IRIS','DCOMIRIS','NOM_IRIS','TYP_IRIS','ORIGINE']].drop_duplicates()
    res.to_csv('Data//Transac_IRIS//res_'+str(i)+'.csv',sep=';')
    print(i)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68


In [61]:
d=pd.read_csv('Data//Transac_IRIS//res_0.csv',sep=';',index_col=0)

for i in range(batch):
    df=pd.read_csv('Data//Transac_IRIS//res_'+str(i)+'.csv',sep=';',index_col=0)
    df=df[['id_transaction','latitude','longitude','geometry','DEPCOM','NOM_COM','IRIS','DCOMIRIS','NOM_IRIS','TYP_IRIS','ORIGINE']].drop_duplicates()
    d=pd.concat([d,df],axis=0) 

d.head()

,id_transaction,latitude,longitude,geometry,DEPCOM,NOM_COM,IRIS,DCOMIRIS,NOM_IRIS,TYP_IRIS,ORIGINE
35884,107332,45.903703,4.814195,POINT (4.814194841969401 45.9037031646598),1238,MASSIEUX,0,12380000,MASSIEUX,Z,NaN
36875,115738,45.910008,4.823275,POINT (4.82327502644876 45.910007805274),1238,MASSIEUX,0,12380000,MASSIEUX,Z,NaN
37586,116451,45.903703,4.814195,POINT (4.814194841969401 45.9037031646598),1238,MASSIEUX,0,12380000,MASSIEUX,Z,NaN
43710,108425,45.908842,4.824928,POINT (4.82492846994613 45.9088416047503),1238,MASSIEUX,0,12380000,MASSIEUX,Z,NaN
56658,16081,45.908842,4.824928,POINT (4.82492846994613 45.9088416047503),1238,MASSIEUX,0,12380000,MASSIEUX,Z,NaN


In [ ]:
transactions_2019=pd.read_csv('transactions_2019.csv',sep=';',index_col=0)
res=transactions_2019.merge(d[['id_transaction','geometry','DEPCOM','NOM_COM','IRIS','DCOMIRIS','NOM_IRIS','TYP_IRIS','ORIGINE']],how='left',on=['id_transaction'])


In [ ]:
#on récupère le nom de l'iris pour le reste du dataset transaction_2019

comp=batch_iris(res[res['NOM_IRIS'].isna()][['id_transaction','date_transaction','prix','departement','id_ville','adresse','ville','code_postal','latitude', 'longitude','type_batiment','vefa','n_pieces','surface_habitable']],geoiris)
comp.drop_duplicates(inplace=True)

comp=comp[['id_transaction', 'date_transaction', 'prix', 'departement', 'id_ville',
       'adresse', 'ville', 'code_postal', 'latitude', 'longitude',
       'type_batiment', 'vefa', 'n_pieces', 'surface_habitable', 'geometry',
       'DEPCOM', 'NOM_COM', 'IRIS', 'DCOMIRIS',
       'NOM_IRIS', 'TYP_IRIS', 'ORIGINE']]


In [ ]:
res_final=pd.concat([res[~(res['NOM_IRIS'].isna())],comp],axis=0)

In [ ]:
res_final.set_index(['id_transaction'],inplace=True)
res_final.to_csv('BD_Transaction_import.csv',sep=';',index=True)

In [108]:
res_final[res_final['NOM_COM']=='GRENOBLE'].reset_index().groupby(['NOM_IRIS']).agg({'id_transaction':'count'}).sort_values(by=['id_transaction'],ascending=False).tail(15)

,id_transaction
NOM_IRIS,
LA BRUYERE,77
ESPLANADE,69
BALADINS,65
TEISSEIRE,54
HELBRONNER-GEANTS,51
JOUHAUX,50
POTERNE,43
PAUL COCAT,43
CONSTANTINE,39
